# CONTESTABLE AUTOMATIC DOWNLOADER SCRIPT 
just edit the x and y dates


In [1]:
import pandas as pd
from glob import glob
import openpyxl as xl
import os
import requests
import datetime
pd.set_option("display.max_columns", None) #show all columns

In [2]:
dates = pd.date_range(start="2023-06-26",end="2023-07-25").strftime('%Y-%m-%d').tolist() #make sure to add end date + 1
dates =[date.replace('-','') for date in dates]
dates =[int(date) for date in dates]
month = ''
len(dates)


30

In [3]:
dates

[20230626,
 20230627,
 20230628,
 20230629,
 20230630,
 20230701,
 20230702,
 20230703,
 20230704,
 20230705,
 20230706,
 20230707,
 20230708,
 20230709,
 20230710,
 20230711,
 20230712,
 20230713,
 20230714,
 20230715,
 20230716,
 20230717,
 20230718,
 20230719,
 20230720,
 20230721,
 20230722,
 20230723,
 20230724,
 20230725]

In [4]:
# #DATE: separated by x and y kasi hassle gumawa ng drop down of dates. Plus, it makes it easier to choose specific intervals
# x = list(range(20201226,20221131)) #date should be from 26th till end of month + 1
# y = list(range(20221201,20221226)) #date should be from 1st to 25th + 1
# dates = x +y
# dates

In [5]:
#Shangrila
for x in dates:
    payload = {'serialno':"213544516", 'datetime': x} #inputs for contestable site
    #r = requests.post('http://billing.akelco.com.ph/mail/ccustomerHC.asp', data=payload) #post
    #r = requests.post('http://192.168.100.4/mail/ccustomerh.asp', data=payload) #post
    r = requests.post('http://billing.akelco.com.ph/mail/ccustomerh.asp', data=payload) #post
    data = pd.read_html(r.text, header = 0)[0]
    data = data.drop(data.index[0]).reset_index()
    data[['DATE', 'TIME']] = data['Date Time'].str.split(' ', 1, expand=True) #split to date and time
    data['TIME'] = data['TIME'].fillna('12:00:00 AM') #fill 12:00:00 no values
    data[['MONTH', 'DAY','YEAR']] = data['DATE'].str.split('/', 2, expand=True) 
    data['DAY'] = data.apply(lambda x:('0' + x['DAY']) if len(x['DAY']) == 1 else x['DAY'], axis=1) #adds '0' for single digit days
    data['MONTH'] = data.apply(lambda x:('0' + x['MONTH']) if len(x['MONTH']) == 1 else x['MONTH'], axis=1) #adds '0' for single digit month
    data[['HOUR', 'MINSECAMPM']] = data['TIME'].str.split(':', 1, expand=True) #split to hour and min
    data[['MINSEC', 'AMPM']] = data['MINSECAMPM'].str.split(' ', 1, expand=True) #split to get AMP/PM
    data[['MIN', 'SEC']] = data['MINSEC'].str.split(':', 1, expand=True) #split to get MIN and SEC
    data['HOUR'] = pd.to_numeric(data['HOUR']) #convert to int
    data['HOUR'] = data.apply(lambda x:x['HOUR']+12 if x['AMPM']== 'PM' else x['HOUR'], axis=1) #convert PM dates to 24hour format
    data['HOUR'] = data['HOUR'].replace(12,0) #replace 12:00(from previous day) to zero hour
    data['HOUR'] = data['HOUR'].replace(24,12) #replace 24:00(12pm +12) back to 12th hour
    data['HOUR'] = data['HOUR'].astype(str) #convert to str
    data['HOUR'] = data.apply(lambda x:('0' + x['HOUR']) if len(x['HOUR']) == 1 else x['HOUR'], axis=1) #adds '0' for single digit hours
    data['SEIL'] = 'R3PNTAKEL07MKSR02'
    fivemin = data[['SEIL']].copy()
    fivemin['BDATE'] = data['YEAR'] + '-' + data['MONTH'] + '-' + data['DAY']
    fivemin['TIME'] = data['HOUR'] + ':' + data['MIN'] 
    fivemin['KW_DEL']= data['Import W Demand Total']/1000
    fivemin['KWH_DEL']= data['Import Wh Total']/1000
    fivemin['KVARH_DEL']= data['Import Varh Total']/1000
    fivemin['KW_REC']= data['Export W Demand Total']/1000
    fivemin['KWH_REC']= data['Export Wh Total']/1000
    fivemin['KVARH_REC']= data['Export Varh Total']/1000
    fivemin['ESTIMATION_FLAG'] = ''
    fivemin['MONTH'] = month
    name5 = 'MET_AKELCOMSP_R3PNTAKEL07MKSR02_'+ data['YEAR'][0]+data['MONTH'][0]+data['DAY'][0]+'.csv'
    fivemin.to_csv(name5,index = False)
#     fifteenmin = fivemin[['SEIL']][0:96]
#     fifteenmin = fifteenmin.rename(columns={'SEIL':'SEIN'})
#     fifteenmin['DATE'] = fivemin['BDATE'].groupby(fivemin.index // 3).max()
#     fifteenmin['TIME'] =  fivemin.loc[2::3]['TIME'].reset_index()['TIME']  #hacky solution to fix NaN due to index slice
#     fifteenmin['KW D']= fivemin['KW_DEL'].groupby(fivemin.index // 3).max()
#     fifteenmin['KWH D']= fivemin['KWH_DEL'].groupby(fivemin.index // 3).sum()
#     fifteenmin['KVARH D']= fivemin['KVARH_DEL'].groupby(fivemin.index // 3).sum()
#     #fiteenmin['KW_REC']= fivemin['KW_REC'].groupby(fivemin.index // 3).max()
#     #fiteenmin['KWH_REC']= fivemin['KWH_REC'].groupby(fivemin.index // 3).max()
#     #fiteenmin['KVARH_REC']= fivemin['KVARH_REC'].groupby(fivemin.index // 3).sum()
#     #fifteenmin.loc[95,'TIME']= '00:00'
#     name15 = 'MET_AKELCOMSP_R3PNTAKEL07MKSR02_'+ data['YEAR'][0]+data['MONTH'][0]+data['DAY'][0]+'.xlsx' #name of excel file for 15min
#     fifteenmin.to_excel('temp.xlsx',sheet_name='R3PNTAKEL07MKSR02',index = False)  #temporary file 
#     path1 = 'temp.xlsx'
#     path2 = "template_15.xlsx"
#     wb1 = xl.load_workbook(filename=path1)
#     ws1 = wb1.worksheets[0]
#     wb2 = xl.load_workbook(filename=path2)
#     ws2 = wb2.worksheets[0]
#     for row in ws1:
#         for cell in row:
#             ws2[cell.coordinate].value = cell.value
#     ss_sheet = wb2["Sheet1"]
#     ss_sheet.title = 'R3PNTAKEL07MKSR02'     #rename sheet 
#     wb2.save(name15)
#     os.remove('temp.xlsx')

In [6]:
#Calveston
for x in dates:
    payload = {'serialno':"213544389", 'datetime': x} #inputs for contestable site
    #r = requests.post('http://billing.akelco.com.ph/mail/ccustomerHC.asp', data=payload) #post
    #r = requests.post('http://192.168.100.4/mail/ccustomerh.asp', data=payload) #post
    r = requests.post('http://billing.akelco.com.ph/mail/ccustomerh.asp', data=payload) #post
    data = pd.read_html(r.text, header = 0)[0]
    data = data.drop(data.index[0]).reset_index()
    data[['DATE', 'TIME']] = data['Date Time'].str.split(' ', 1, expand=True) #split to date and time
    data['TIME'] = data['TIME'].fillna('12:00:00 AM') #fill 12:00:00 no values
    data[['MONTH', 'DAY','YEAR']] = data['DATE'].str.split('/', 2, expand=True) 
    data['DAY'] = data.apply(lambda x:('0' + x['DAY']) if len(x['DAY']) == 1 else x['DAY'], axis=1) #adds '0' for single digit days
    data['MONTH'] = data.apply(lambda x:('0' + x['MONTH']) if len(x['MONTH']) == 1 else x['MONTH'], axis=1) #adds '0' for single digit month
    data[['HOUR', 'MINSECAMPM']] = data['TIME'].str.split(':', 1, expand=True) #split to hour and min
    data[['MINSEC', 'AMPM']] = data['MINSECAMPM'].str.split(' ', 1, expand=True) #split to get AMP/PM
    data[['MIN', 'SEC']] = data['MINSEC'].str.split(':', 1, expand=True) #split to get MIN and SEC
    data['HOUR'] = pd.to_numeric(data['HOUR']) #convert to int
    data['HOUR'] = data.apply(lambda x:x['HOUR']+12 if x['AMPM']== 'PM' else x['HOUR'], axis=1) #convert PM dates to 24hour format
    data['HOUR'] = data['HOUR'].replace(12,0) #replace 12:00(from previous day) to zero hour
    data['HOUR'] = data['HOUR'].replace(24,12) #replace 24:00(12pm +12) back to 12th hour
    data['HOUR'] = data['HOUR'].astype(str) #convert to str
    data['HOUR'] = data.apply(lambda x:('0' + x['HOUR']) if len(x['HOUR']) == 1 else x['HOUR'], axis=1) #adds '0' for single digit hours
    data['SEIL'] = 'R3PNTAKEL07CLVST1'
    fivemin = data[['SEIL']].copy()
    fivemin['BDATE'] = data['YEAR'] + '-' + data['MONTH'] + '-' + data['DAY']
    fivemin['TIME'] = data['HOUR'] + ':' + data['MIN'] 
    fivemin['KW_DEL']= data['Import W Demand Total']/1000
    fivemin['KWH_DEL']= data['Import Wh Total']/1000
    fivemin['KVARH_DEL']= data['Import Varh Total']/1000
    fivemin['KW_REC']= data['Export W Demand Total']/1000
    fivemin['KWH_REC']= data['Export Wh Total']/1000
    fivemin['KVARH_REC']= data['Export Varh Total']/1000
    fivemin['ESTIMATION_FLAG'] = ''
    fivemin['MONTH'] = month
    name5 = 'MET_AKELCOMSP_R3PNTAKEL07CLVST1_'+ data['YEAR'][0]+data['MONTH'][0]+data['DAY'][0]+'.csv'
    fivemin.to_csv(name5,index = False)
#     fifteenmin = fivemin[['SEIL']][0:96]
#     fifteenmin = fifteenmin.rename(columns={'SEIL':'SEIN'})
#     fifteenmin['DATE'] = fivemin['BDATE'].groupby(fivemin.index // 3).max()
#     fifteenmin['TIME'] =  fivemin.loc[2::3]['TIME'].reset_index()['TIME']  #hacky solution to fix NaN due to index slice
#     fifteenmin['KW D']= fivemin['KW_DEL'].groupby(fivemin.index // 3).max()
#     fifteenmin['KWH D']= fivemin['KWH_DEL'].groupby(fivemin.index // 3).sum()
#     fifteenmin['KVARH D']= fivemin['KVARH_DEL'].groupby(fivemin.index // 3).sum()
#     #fiteenmin['KW_REC']= fivemin['KW_REC'].groupby(fivemin.index // 3).max()
#     #fiteenmin['KWH_REC']= fivemin['KWH_REC'].groupby(fivemin.index // 3).max()
#     #fiteenmin['KVARH_REC']= fivemin['KVARH_REC'].groupby(fivemin.index // 3).sum()
#     #fifteenmin.loc[95,'TIME']= '00:00'
#     name15 = 'MET_AKELCOMSP_R3PNTAKEL07CLVST1_'+ data['YEAR'][0]+data['MONTH'][0]+data['DAY'][0]+'.xlsx'
#     fifteenmin.to_excel('temp.xlsx',sheet_name='R3PNTAKEL07CLVST1',index = False)
#     path1 = 'temp.xlsx'
#     path2 = "template_15.xlsx"
#     wb1 = xl.load_workbook(filename=path1)
#     ws1 = wb1.worksheets[0]
#     wb2 = xl.load_workbook(filename=path2)
#     ws2 = wb2.worksheets[0]
#     for row in ws1:
#         for cell in row:
#             ws2[cell.coordinate].value = cell.value
#     ss_sheet = wb2["Sheet1"]
#     ss_sheet.title = 'R3PNTAKEL07CLVST1'
#     wb2.save(name15)
#     os.remove('temp.xlsx')

In [7]:
#New Coast
for x in dates:
    payload = {'serialno':"213544390", 'datetime': x} #inputs for contestable site
    #r = requests.post('http://billing.akelco.com.ph/mail/ccustomerHC.asp', data=payload) #post
    #r = requests.post('http://192.168.100.4/mail/ccustomerh.asp', data=payload) #post
    r = requests.post('http://billing.akelco.com.ph/mail/ccustomerh.asp', data=payload) #post
    data = pd.read_html(r.text, header = 0)[0]
    data = data.drop(data.index[0]).reset_index()
    data[['DATE', 'TIME']] = data['Date Time'].str.split(' ', 1, expand=True) #split to date and time
    data['TIME'] = data['TIME'].fillna('12:00:00 AM') #fill 12:00:00 no values
    data[['MONTH', 'DAY','YEAR']] = data['DATE'].str.split('/', 2, expand=True) 
    data['DAY'] = data.apply(lambda x:('0' + x['DAY']) if len(x['DAY']) == 1 else x['DAY'], axis=1) #adds '0' for single digit days
    data['MONTH'] = data.apply(lambda x:('0' + x['MONTH']) if len(x['MONTH']) == 1 else x['MONTH'], axis=1) #adds '0' for single digit month
    data[['HOUR', 'MINSECAMPM']] = data['TIME'].str.split(':', 1, expand=True) #split to hour and min
    data[['MINSEC', 'AMPM']] = data['MINSECAMPM'].str.split(' ', 1, expand=True) #split to get AMP/PM
    data[['MIN', 'SEC']] = data['MINSEC'].str.split(':', 1, expand=True) #split to get MIN and SEC
    data['HOUR'] = pd.to_numeric(data['HOUR']) #convert to int
    data['HOUR'] = data.apply(lambda x:x['HOUR']+12 if x['AMPM']== 'PM' else x['HOUR'], axis=1) #convert PM dates to 24hour format
    data['HOUR'] = data['HOUR'].replace(12,0) #replace 12:00(from previous day) to zero hour
    data['HOUR'] = data['HOUR'].replace(24,12) #replace 24:00(12pm +12) back to 12th hour
    data['HOUR'] = data['HOUR'].astype(str) #convert to str
    data['HOUR'] = data.apply(lambda x:('0' + x['HOUR']) if len(x['HOUR']) == 1 else x['HOUR'], axis=1) #adds '0' for single digit hours
    data['SEIL'] = 'R3PNTAKEL07GERI01'
    fivemin = data[['SEIL']].copy()
    fivemin['BDATE'] = data['YEAR'] + '-' + data['MONTH'] + '-' + data['DAY']
    fivemin['TIME'] = data['HOUR'] + ':' + data['MIN'] 
    fivemin['KW_DEL']= data['Import W Demand Total']/1000
    fivemin['KWH_DEL']= data['Import Wh Total']/1000
    fivemin['KVARH_DEL']= data['Import Varh Total']/1000
    fivemin['KW_REC']= data['Export W Demand Total']/1000
    fivemin['KWH_REC']= data['Export Wh Total']/1000
    fivemin['KVARH_REC']= data['Export Varh Total']/1000
    fivemin['ESTIMATION_FLAG'] = ''
    fivemin['MONTH'] = month
    name5 = 'MET_AKELCOMSP_R3PNTAKEL07GERI01_'+ data['YEAR'][0]+data['MONTH'][0]+data['DAY'][0]+'.csv'
    fivemin.to_csv(name5,index = False)
#     fifteenmin = fivemin[['SEIL']][0:96]
#     fifteenmin = fifteenmin.rename(columns={'SEIL':'SEIN'})
#     fifteenmin['DATE'] = fivemin['BDATE'].groupby(fivemin.index // 3).max()
#     fifteenmin['TIME'] =  fivemin.loc[2::3]['TIME'].reset_index()['TIME']  #hacky solution to fix NaN due to index slice
#     fifteenmin['KW D']= fivemin['KW_DEL'].groupby(fivemin.index // 3).max()
#     fifteenmin['KWH D']= fivemin['KWH_DEL'].groupby(fivemin.index // 3).sum()
#     fifteenmin['KVARH D']= fivemin['KVARH_DEL'].groupby(fivemin.index // 3).sum()
#     #fiteenmin['KW_REC']= fivemin['KW_REC'].groupby(fivemin.index // 3).max()
#     #fiteenmin['KWH_REC']= fivemin['KWH_REC'].groupby(fivemin.index // 3).max()
#     #fiteenmin['KVARH_REC']= fivemin['KVARH_REC'].groupby(fivemin.index // 3).sum()
#     #fifteenmin.loc[95,'TIME']= '00:00'
#     name15 = 'MET_AKELCOMSP_R3PNTAKEL07GERI01_'+ data['YEAR'][0]+data['MONTH'][0]+data['DAY'][0]+'.xlsx'
#     fifteenmin.to_excel('temp.xlsx',sheet_name='R3PNTAKEL07GERI01',index = False)
#     path1 = 'temp.xlsx'
#     path2 = "template_15.xlsx"
#     wb1 = xl.load_workbook(filename=path1)
#     ws1 = wb1.worksheets[0]
#     wb2 = xl.load_workbook(filename=path2)
#     ws2 = wb2.worksheets[0]
#     for row in ws1:
#         for cell in row:
#             ws2[cell.coordinate].value = cell.value
#     ss_sheet = wb2["Sheet1"]
#     ss_sheet.title = 'R3PNTAKEL07GERI01'
#     wb2.save(name15)
#     os.remove('temp.xlsx')

In [8]:
    payload = {'serialno':"213544390", 'datetime': x} #inputs for contestable site
    #r = requests.post('http://billing.akelco.com.ph/mail/ccustomerHC.asp', data=payload) #post
    #r = requests.post('http://192.168.100.4/mail/ccustomerh.asp', data=payload) #post
    r = requests.post('http://billing.akelco.com.ph/mail/ccustomerh.asp', data=payload) #post
    data = pd.read_html(r.text, header = 0)[0]
    data = data.drop(data.index[0]).reset_index()
    data[['DATE', 'TIME']] = data['Date Time'].str.split(' ', 1, expand=True) #split to date and time
    data['TIME'] = data['TIME'].fillna('12:00:00 AM') #fill 12:00:00 no values
    data[['MONTH', 'DAY','YEAR']] = data['DATE'].str.split('/', 2, expand=True) 
    data['DAY'] = data.apply(lambda x:('0' + x['DAY']) if len(x['DAY']) == 1 else x['DAY'], axis=1) #adds '0' for single digit days
    data['MONTH'] = data.apply(lambda x:('0' + x['MONTH']) if len(x['MONTH']) == 1 else x['MONTH'], axis=1) #adds '0' for single digit month
    data[['HOUR', 'MINSECAMPM']] = data['TIME'].str.split(':', 1, expand=True) #split to hour and min
    data[['MINSEC', 'AMPM']] = data['MINSECAMPM'].str.split(' ', 1, expand=True) #split to get AMP/PM
    data[['MIN', 'SEC']] = data['MINSEC'].str.split(':', 1, expand=True) #split to get MIN and SEC
    data['HOUR'] = pd.to_numeric(data['HOUR']) #convert to int
    data['HOUR'] = data.apply(lambda x:x['HOUR']+12 if x['AMPM']== 'PM' else x['HOUR'], axis=1) #convert PM dates to 24hour format
    data['HOUR'] = data['HOUR'].replace(12,0) #replace 12:00(from previous day) to zero hour
    data['HOUR'] = data['HOUR'].replace(24,12) #replace 24:00(12pm +12) back to 12th hour
    data['HOUR'] = data['HOUR'].astype(str) #convert to str
    data['HOUR'] = data.apply(lambda x:('0' + x['HOUR']) if len(x['HOUR']) == 1 else x['HOUR'], axis=1) #adds '0' for single digit hours
    data['SEIL'] = 'R3PNTAKEL07GERI01'
    fivemin = data[['SEIL']].copy()
    fivemin['BDATE'] = data['YEAR'] + '-' + data['MONTH'] + '-' + data['DAY']
    fivemin['TIME'] = data['HOUR'] + ':' + data['MIN'] 
    fivemin['KW_DEL']= data['Import W Demand Total']/1000
    fivemin['KWH_DEL']= data['Import Wh Total']/1000
    fivemin['KVARH_DEL']= data['Import Varh Total']/1000
    fivemin['KW_REC']= data['Export W Demand Total']/1000
    fivemin['KWH_REC']= data['Export Wh Total']/1000
    fivemin['KVARH_REC']= data['Export Varh Total']/1000
    fivemin['ESTIMATION_FLAG'] = ''
    fivemin['MONTH'] = month
    name5 = 'MET_AKELCOMSP_R3PNTAKEL07GERI01_'+ data['YEAR'][0]+data['MONTH'][0]+data['DAY'][0]+'.csv'
    fivemin.to_csv(name5,index = False)
#     fifteenmin = fivemin[['SEIL']][0:96]
#     fifteenmin = fifteenmin.rename(columns={'SEIL':'SEIN'})
#     fifteenmin['DATE'] = fivemin['BDATE'].groupby(fivemin.index // 3).max()
#     fifteenmin['TIME'] =  fivemin.loc[2::3]['TIME'].reset_index()['TIME']  #hacky solution to fix NaN due to index slice
#     fifteenmin['KW D']= fivemin['KW_DEL'].groupby(fivemin.index // 3).max()
#     fifteenmin['KWH D']= fivemin['KWH_DEL'].groupby(fivemin.index // 3).sum()
#     fifteenmin['KVARH D']= fivemin['KVARH_DEL'].groupby(fivemin.index // 3).sum()
#     #fiteenmin['KW_REC']= fivemin['KW_REC'].groupby(fivemin.index // 3).max()
#     #fiteenmin['KWH_REC']= fivemin['KWH_REC'].groupby(fivemin.index // 3).max()
#     #fiteenmin['KVARH_REC']= fivemin['KVARH_REC'].groupby(fivemin.index // 3).sum()
#     #fifteenmin.loc[95,'TIME']= '00:00'
#     name15 = 'MET_AKELCOMSP_R3PNTAKEL07GERI01_'+ data['YEAR'][0]+data['MONTH'][0]+data['DAY'][0]+'.xlsx'
#     fifteenmin.to_excel('temp.xlsx',sheet_name='R3PNTAKEL07GERI01',index = False)
#     path1 = 'temp.xlsx'
#     path2 = "template_15.xlsx"
#     wb1 = xl.load_workbook(filename=path1)
#     ws1 = wb1.worksheets[0]
#     wb2 = xl.load_workbook(filename=path2)
#     ws2 = wb2.worksheets[0]
#     for row in ws1:
#         for cell in row:
#             ws2[cell.coordinate].value = cell.value
#     ss_sheet = wb2["Sheet1"]
#     ss_sheet.title = 'R3PNTAKEL07GERI01'
#     wb2.save(name15)
#     os.remove('temp.xlsx')